# S.3 Period finding

Shor's algorithm is characterized by having a classical part and a quantum part. In particular, the quantum part is in charge of searching efficiently for nontrivial square roots using an algorithm called period finding. 

![](https://codebook.xanadu.ai/pics/PF_circuit.jpg)

### Codercise S.3.1. 
Implement the above circuit for
. We will make use of PennyLane's get_unitary_matrix function which, given a quantum function, returns the associated matrix. Finally, we will use the get_phase function to translate the output to a decimal value.

In [ ]:
def U():
    qml.SWAP(wires=[2,3])
    qml.SWAP(wires=[1,2])
    qml.SWAP(wires=[0,1])
    for i in range(4):
        qml.PauliX(wires=i)

matrix = get_unitary_matrix(U, wire_order=range(4))()

n_target_wires = 4
target_wires = range(n_target_wires)
n_estimation_wires = 3
estimation_wires = range(4, 4 + n_estimation_wires)


dev = qml.device("default.qubit", shots=1, wires=n_target_wires+n_estimation_wires)

@qml.qnode(dev)
def circuit(matrix):
    """Return a sample after taking a shot at the estimation wires.
    
    Args:
        matrix (array[complex]): matrix representation of U.

    Returns:
        array[float]: a sample after taking a shot at the estimation wires.
    """
    
    ##################
    # YOUR CODE HERE #
    ##################
    
    # CREATE THE INITIAL STATE |0001> ON TARGET WIRES
    qml.PauliX(wires=3)
    # USE THE SUBROUTINE QUANTUM PHASE ESTIMATION
    qml.QuantumPhaseEstimation(matrix,[0,1,2,3],[4,5,6])
    
    return qml.sample(wires=estimation_wires)

def get_phase(matrix):
    binary = "".join([str(b) for b in circuit(matrix)])
    return int(binary, 2) / 2 ** n_estimation_wires

for i in range(5):
    print(circuit(matrix))
    print(f"shot {i+1}, phase:",get_phase(matrix))


### Codercise S.3.2. 
Assuming that we have the get_phase function, obtain the value of the period with the procedure explained above. Help yourself to the function Fraction (it is already imported for you).

In [ ]:
def U():
    qml.SWAP(wires=[2,3])
    qml.SWAP(wires=[1,2])
    qml.SWAP(wires=[0,1])
    for i in range(4):
        qml.PauliX(wires=i)

matrix = get_unitary_matrix(U, wire_order=range(4))()

target_wires = range(4)
n_estimation_wires = 3
estimation_wires = range(4, 4 + n_estimation_wires)

def get_period(matrix):
    """Return the period of the state using the already-defined 
    get_phase function.
    
    Args:
        matrix (array[complex]): matrix associated with the operator U
        
    Returns:
        int: Obtained period of the state.
    """
    
    shots = 10
    
    ##################
    # YOUR CODE HERE #
    ##################
    arr = []
    for _ in range(shots):
        x = get_phase(matrix)
        f = Fraction(x).limit_denominator(2**3)
        
        if x==0:
            continue
        else:
            arr.append(f.denominator)
    lcm =1
    for _ in arr:
        lcm = lcm*_//np.gcd(lcm,_)
    return lcm
    

    

print(get_period(matrix))


### Codercise S.3.3. 
Create a circuit in which the operator is executed four times on the state
to check that the results obtained in the previous exercise are consistent.

In [ ]:
def U():
    qml.SWAP(wires=[2,3])
    qml.SWAP(wires=[1,2])
    qml.SWAP(wires=[0,1])
    for i in range(4):
        qml.PauliX(wires=i)

dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def circuit():
    """Apply U four times to |0001> to verify this is the period.
    
    Returns:
        array[float]: probabilities of each basis state. 
    """
    
    ##################
    # YOUR CODE HERE #
    ##################
    qml.PauliX(wires=3)
    U()
    U()
    U()
    U()
    
    
    return qml.probs(wires=range(4))


print(circuit())
